<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/europython-2022/03-gradio-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Transformers demo with Gradio

## References

* https://huggingface.co/blog/gradio-spaces
* https://huggingface.co/blog/gradio

In [ ]:
# !pip install transformers gradio sentencepiece

## Example 1: Using the Transformers pipeline

In [ ]:
import gradio as gr
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="Rocketknight1/europython-imdb")

In [ ]:
pipe("This was the most boring 94 minutes of my life.")

In [ ]:
label2emoji = {"Negative": "💩", "Positive": "😻"}

def predict(text):
    preds = pipe(text)[0]
    return label2emoji[preds["label"]], round(preds["score"], 5)

predict("I loved this movie!")

In [ ]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Review analysis",
    description="Enter some review text and check what the model predicts for its sentiment.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["I loved it, best movie ever!"], ["Turgid and preposterous, almost unwatchable."]
    ],
)

gradio_ui.launch(debug=True)

## Example 2: Using the Inference API from the Hugging Face Hub

In [ ]:
from huggingface_hub import InferenceApi

text = "I loved it, best movie ever!"
inference = InferenceApi("Rocketknight1/europython-imdb")
preds = inference(inputs=text)
preds[0]

In [ ]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

In [ ]:
def inference_predict(text):
    inference = InferenceApi("Rocketknight1/europython-imdb")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [ ]:
inference_predict(text)

In [ ]:
gradio_ui = gr.Interface.load(
    name="Rocketknight1/europython-imdb",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some review text and check what the model predicts for its sentiment.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["I loved it, best movie ever!"], ["Turgid and preposterous, almost unwatchable."]
    ],
)

gradio_ui.launch(debug=True)